In [1]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

train_data = pd.read_csv("dev/train_features.csv")
train_metadata = pd.read_csv("dev/train_metadata.csv")
ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ana)
ana.plot_data()


MachineLearningAnalyses 
  variables: 20 rows, 13 columns
  metadata: 20 rows, 14 columns



In [2]:
import numpy as np
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scl = MachineLearningScaleFeaturesScalerSklearn(scaler_type="MaxAbsScaler")
iso = MachineLearningMethodIsolationForestSklearn(random_state = 30, contamination = "auto")

#scales and fits the model and upgrades analyses class to IsolationForestAnalyses
ana = scl.run(ana)
ana = iso.run(ana)
print(ana.__class__)

ana.plot_data()

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



<class 'src.StreamPort.machine_learning.analyses.IsolationForestAnalyses'>


In [3]:
fig_train_scores=ana.plot_scores()
fig_train_scores.write_image("dev/figures/fig_train_scores.png", width=1100, height= 350, scale = 3)
fig_train_scores.show()

In [4]:
test_data = pd.read_csv("dev/test_features.csv")
test_metadata = pd.read_csv("dev/test_metadata.csv")

ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)

fig_test_scores = ana.plot_scores()
fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1100, height= 350, scale = 3)
fig_test_scores.show()

outliers_test["outlier"] = outliers_test["outlier"].map({True: "outlier", False: "normal"})
outliers_test["class"] = outliers_test["outlier"]
outliers_test = outliers_test.drop(columns=["outlier"])
if "dev/test_metadata_classified.csv":
    test_metadata = pd.read_csv("dev/test_metadata_classified.csv")  
else:
    test_metadata = pd.concat([test_metadata, outliers_test], axis=1)
test_metadata.to_csv("dev/test_metadata_classified.csv", index=False)

    outlier     score
0      True -0.164191
1      True -0.109980
2      True -0.087441
3      True -0.111057
4      True -0.099810
5      True -0.108168
6     False -0.051946
7      True -0.113809
8      True -0.094775
9      True -0.112058
10     True -0.108988
11    False -0.061363
12     True -0.092697
13     True -0.090416
14     True -0.086869
15    False -0.079516
16    False -0.080108


Addition of new training data based on previous prediction results

In [5]:
#ana.add_prediction()
#print(ana)

In [6]:
fig_test_features = ana.plot_data()
fig_test_features.write_image("dev/figures/fig_test_features.png", width=1100, height= 350, scale = 3)
fig_test_features.show()

In [7]:
test2_data = pd.read_csv("dev/test2_features.csv")
test2_metadata = pd.read_csv("dev/test2_metadata.csv")
ana.predict(test2_data, test2_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
ana.plot_scores()

   outlier     score
0     True -0.164191
1     True -0.159730
2     True -0.140807
3     True -0.111057
4     True -0.126519


In [8]:
#ana.add_prediction()
#print(ana)

In [9]:
fig_test2_features = ana.plot_data()
fig_test2_features.write_image("dev/figures/fig_test2_features.png", width=1100, height= 350, scale = 3)
fig_test2_features.show()

In [10]:
# import shap
# from src.StreamPort.machine_learning.methods import MachineLearningExplainModelShap

# shap_explainer = MachineLearningExplainModelShap(model = "classification", model_type = "tree")
# ana = shap_explainer.run(ana)
# shap_values = ana.data["shap_values"]
# print(shap_values)
#shap.summary_plot(shap_values, test2_data)

In [ ]:
for i in [3, 4, 5, 6, 7]:
    test_data = pd.read_csv(f"dev/test{i}_features.csv")
    test_metadata = pd.read_csv(f"dev/test{i}_metadata.csv")
    ana.predict(test_data, test_metadata)
    outliers_test = ana.test_prediction_outliers()
    print("Test set ", i, " indices: \n", test_metadata["index"])
    print(outliers_test)
    fig_test_scores = ana.plot_scores()
    fig_test_scores.write_image(f"dev/figures/fig_test{i}_scores.png", width=1100, height= 350, scale = 3)
    fig_test_scores.show()
    #ana.add_prediction()
    print(ana)
    fig_test_features = ana.plot_data()
    fig_test_features.write_image(f"dev/figures/fig_test{i}_features.png", width=1100, height= 350, scale = 3)
    fig_test_features.show()

Test set  3 : indices:  0    234
1    118
2    153
3    240
4    276
5    160
6    149
7    278
8    125
9    158
Name: index, dtype: int64
   outlier     score
0     True -0.100146
1    False -0.015745
2    False -0.055556
3     True -0.110838
4     True -0.094544
5    False -0.047755
6     True -0.084618
7     True -0.099756
8    False -0.027466
9    False -0.062511



IsolationForestAnalyses 
  variables: 20 rows, 13 columns
  metadata: 20 rows, 14 columns



Test set  4 : indices:  0    152
1    156
2    276
3    116
4    120
5    165
6    267
7    157
8    244
9    242
Name: index, dtype: int64
   outlier     score
0    False -0.037123
1    False -0.071201
2     True -0.094544
3     True -0.153536
4    False -0.044035
5     True -0.085806
6    False -0.027148
7    False -0.055440
8     True -0.113190
9     True -0.084423



IsolationForestAnalyses 
  variables: 20 rows, 13 columns
  metadata: 20 rows, 14 columns



Test set  5 : indices:  0     50
1    276
2    247
3    237
4    242
5    277
6    129
7    153
8    116
9    270
Name: index, dtype: int64
   outlier     score
0    False  0.008704
1     True -0.094544
2     True -0.119627
3     True -0.100230
4     True -0.084423
5     True -0.089628
6    False -0.058689
7    False -0.055556
8     True -0.153536
9     True -0.086842



IsolationForestAnalyses 
  variables: 20 rows, 13 columns
  metadata: 20 rows, 14 columns



Test set  6 : indices:  0     49
1    269
2    278
3    268
4    277
5    118
6    238
7    155
8    244
9    128
Name: index, dtype: int64
   outlier     score
0    False -0.068943
1    False -0.072416
2     True -0.099756
3    False -0.074009
4     True -0.089628
5    False -0.015745
6     True -0.087119
7    False -0.054006
8     True -0.113190
9    False -0.017167



IsolationForestAnalyses 
  variables: 20 rows, 13 columns
  metadata: 20 rows, 14 columns



Test set  7 : indices:  0    270
1    165
2    236
3    121
4     50
5     16
6    234
7    168
8    273
9     17
Name: index, dtype: int64
   outlier     score
0     True -0.086842
1     True -0.085806
2     True -0.095769
3    False -0.049139
4    False  0.008704
5     True -0.126519
6     True -0.100146
7    False -0.040634
8    False -0.043908
9     True -0.146062



IsolationForestAnalyses 
  variables: 20 rows, 13 columns
  metadata: 20 rows, 14 columns



In [12]:
threshold_record = pd.read_csv("dev/threshold_record.csv", index_col=0) if "dev/threshold_record.csv" else None
import plotly.graph_objects as go

fig = go.Figure()
if threshold_record is not None:
    fig.add_trace(
        go.Scatter(
            x=threshold_record["training set"],
            y=threshold_record["threshold"],
            mode="lines+markers",
            name="Threshold",
            text=threshold_record.index,
            hovertemplate=["<br>Trained on: " + threshold_record.index[i]
                            + "<br>Threshold: " + str(threshold_record["threshold"][i]) 
                            + "<br>Outliers in test set: " + str(threshold_record["outliers"][i]) 
                            + "<br>Outliers %: " + str(threshold_record["outliers %"][i]) + "%" for i in range(len(threshold_record))
                            ],
            line=dict(color="red", width=2, dash='dash'),
            marker=dict(size=8, symbol="circle")
        )
    )
fig.update_layout(
    title="Threshold Increase Over Size of Training Set",
    xaxis_title="Number of Training Curves",
    yaxis_title="Threshold",
    template="plotly_white"
)

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_10768\3255632709.py:14: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_10768\3255632709.py:15: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Sandeep\AppData\Local\Temp\ipykernel_10768\3255632709.py:16: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

